# Creación de subconjuntos

Descargamos los metadatos y etiquetas del conjunto ISIC 2019 para seleccionar 2 subconjuntos balanceados por clase:
- Subconjunto de entrenamiento: 100 imágenes por cada clase
- Subconjunto de test: 10 imágenes por cada clase

El dataset ISIC 2019 contiene 8 clases de lesiones de piel. Luego descargamos el archivo ZIP completo pero solo extraemos las imágenes seleccionadas, evitando problemas de espacio en el GitHub Codespace de 10GB.

## 1. Descarga de metadatos y etiquetas

In [4]:
import pandas as pd
import requests
from pathlib import Path

dest_dir = Path("dataset")
dest_dir.mkdir(parents=True, exist_ok=True)

csv_url = "https://isic-archive.s3.amazonaws.com/challenges/2019/ISIC_2019_Training_Metadata.csv"
gt_url = "https://isic-archive.s3.amazonaws.com/challenges/2019/ISIC_2019_Training_GroundTruth.csv"

csv_path = dest_dir / "ISIC_2019_Training_Metadata.csv"
gt_path = dest_dir / "ISIC_2019_Training_GroundTruth.csv"

with requests.get(csv_url, timeout=60) as r:
    r.raise_for_status()
    csv_path.write_bytes(r.content)

with requests.get(gt_url, timeout=60) as r:
    r.raise_for_status()
    gt_path.write_bytes(r.content)

df_meta = pd.read_csv(csv_path)
df_gt = pd.read_csv(gt_path)

df = pd.merge(df_meta, df_gt, on='image')

print(f"Total de imágenes: {len(df)}")
print(f"Columnas: {list(df.columns)}\n")

clases = [col for col in df_gt.columns if col != 'image']
print(f"Clases de lesiones: {clases}\n")

for clase in clases:
    count = df[clase].sum()
    print(f"{clase}: {count} imágenes")

Total de imágenes: 25331
Columnas: ['image', 'age_approx', 'anatom_site_general', 'lesion_id', 'sex', 'MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']

Clases de lesiones: ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']

MEL: 4522.0 imágenes
NV: 12875.0 imágenes
BCC: 3323.0 imágenes
AK: 867.0 imágenes
BKL: 2624.0 imágenes
DF: 239.0 imágenes
VASC: 253.0 imágenes
SCC: 628.0 imágenes
UNK: 0.0 imágenes


In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

clases = [col for col in df.columns if col not in ['image', 'age_approx', 'anatom_site_general', 'lesion_id', 'sex']]

df['clase'] = df[clases].idxmax(axis=1)

print(f"Distribución de clases:")
print(df['clase'].value_counts())

train_dfs = []
test_dfs = []

for clase in df['clase'].unique():
    df_clase = df[df['clase'] == clase]
    
    if len(df_clase) < 110:
        print(f"Advertencia: {clase} tiene solo {len(df_clase)} imágenes, usando todas disponibles")
        n_train = min(100, int(len(df_clase) * 0.9))
        n_test = len(df_clase) - n_train
    else:
        n_train = 100
        n_test = 10
    
    df_clase_shuffled = df_clase.sample(frac=1, random_state=42).reset_index(drop=True)
    
    train_dfs.append(df_clase_shuffled.iloc[:n_train])
    test_dfs.append(df_clase_shuffled.iloc[n_train:n_train + n_test])

train_df = pd.concat(train_dfs, ignore_index=True)
test_df = pd.concat(test_dfs, ignore_index=True)

train_df.to_csv(dest_dir / "train_subset.csv", index=False)
test_df.to_csv(dest_dir / "test_subset.csv", index=False)

print(f"\nConjunto Train: {len(train_df)} imágenes")
print(train_df['clase'].value_counts())
print(f"\nConjunto Test: {len(test_df)} imágenes")
print(test_df['clase'].value_counts())

train_imgs = set(train_df['image'])
test_imgs = set(test_df['image'])

overlap = train_imgs & test_imgs
if overlap:
    print(f"\nERROR: {len(overlap)} imágenes repetidas entre train y test")
else:
    print("\n✓ No hay solapamiento entre train y test")

all_imgs_sets = {
    'train': train_imgs,
    'test': test_imgs
}

total_images = len(train_imgs) + len(test_imgs)
print(f"\nTotal de imágenes a descargar: {total_images}")

Distribución de clases:
clase
NV      12875
MEL      4522
BCC      3323
BKL      2624
AK        867
SCC       628
VASC      253
DF        239
Name: count, dtype: int64

Conjunto Train: 800 imágenes
clase
NV      100
MEL     100
BKL     100
DF      100
SCC     100
BCC     100
VASC    100
AK      100
Name: count, dtype: int64

Conjunto Test: 80 imágenes
clase
NV      10
MEL     10
BKL     10
DF      10
SCC     10
BCC     10
VASC    10
AK      10
Name: count, dtype: int64

✓ No hay solapamiento entre train y test

Total de imágenes a descargar: 880


## 2. Descarga selectiva del ZIP

In [6]:
import zipfile
import io
import tempfile

zip_url = "https://isic-archive.s3.amazonaws.com/challenges/2019/ISIC_2019_Training_Input.zip"

print("Descargando el archivo ZIP completo...")
print("(Esto puede tardar varios minutos, ~9-10 GB)\n")

# Descarga con streaming a archivo temporal
with tempfile.NamedTemporaryFile(delete=False, suffix='.zip') as tmp_file:
    tmp_path = Path(tmp_file.name)
    
    with requests.get(zip_url, stream=True, timeout=600) as r:
        r.raise_for_status()
        total_size = int(r.headers.get('content-length', 0))
        downloaded = 0
        
        for chunk in r.iter_content(chunk_size=1024*1024):  # 1 MB
            if chunk:
                tmp_file.write(chunk)
                downloaded += len(chunk)
                if total_size > 0:
                    pct = (downloaded / total_size) * 100
                    print(f"Descargado: {downloaded / (1024**3):.2f} GB / {total_size / (1024**3):.2f} GB ({pct:.1f}%)", end='\r')
    
    print("\n\n✓ Descarga completa. Extrayendo imágenes a carpetas separadas...\n")
    
    for subset_name, imgs_set in all_imgs_sets.items():
        subset_dir = dest_dir / subset_name
        subset_dir.mkdir(exist_ok=True)
        
        extracted = 0
        not_found = []
        
        with zipfile.ZipFile(tmp_path, 'r') as z:
            all_files = z.namelist()
            
            for img_name in imgs_set:
                matching = [f for f in all_files if img_name in f and f.endswith('.jpg')]
                
                if matching:
                    for file_path in matching:
                        member = z.getinfo(file_path)
                        member.filename = img_name + '.jpg'
                        z.extract(member, subset_dir)
                        extracted += 1
                else:
                    not_found.append(img_name)
            
            print(f"{subset_name}: {extracted} imágenes extraídas" + (f", {len(not_found)} no encontradas" if not_found else ""))

# Limpia el archivo temporal
tmp_path.unlink(missing_ok=True)

Descargando el archivo ZIP completo...
(Esto puede tardar varios minutos, ~9-10 GB)

Descargado: 9.10 GB / 9.10 GB (100.0%)

✓ Descarga completa. Extrayendo imágenes a carpetas separadas...

train: 800 imágenes extraídas
test: 80 imágenes extraídas
